In [826]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import xgboost as xgb

In [827]:
train_peptides=pd.read_csv(r"C:\Users\jshri\OneDrive\Desktop\train_peptides.csv")
train_clinical_data=pd.read_csv(r"C:\Users\jshri\OneDrive\Desktop\train_clinical_data.csv")

train_peptides.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [828]:
class config:
    MAX_PEPTIDES = 20
    PROFILE_REPORT = True

In [829]:
from sklearn.preprocessing import LabelEncoder

train_peptides.loc[:, "UniProt"] = train_peptides.loc[:, "UniProt"].fillna("None")
train_peptides.loc[:, "PeptideAbundance"] = train_peptides.loc[:, "PeptideAbundance"].fillna(0)
le = LabelEncoder()
train_peptides['UniProt'] = le.fit_transform(train_peptides['UniProt'])
train_peptides.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,0,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,1,GNPEPTFSWTK,102060.0
2,55_0,0,55,1,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,1,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,1,SMEQNGPGLEYR,30838.7


In [830]:
peptides_aux_1 = pd.DataFrame(train_peptides.groupby(by=["patient_id", "visit_id", "visit_month"])["UniProt"].apply(list))#.reset_index()
peptides_aux_2 = pd.DataFrame(train_peptides.groupby(by=["patient_id", "visit_id", "visit_month"])["PeptideAbundance"].apply(list))#.reset_index()
peptides_aux_3 = pd.DataFrame(peptides_aux_1.UniProt.values.tolist()).add_prefix('UniProt_')
peptides_aux_4 = pd.DataFrame(peptides_aux_2.PeptideAbundance.values.tolist()).add_prefix('PeptideAbundance_')
peptides_aux_3.index = peptides_aux_1.index
peptides_aux_4.index = peptides_aux_2.index
new_train_peptides = peptides_aux_3.merge(peptides_aux_4, left_index=True, right_index=True, how="inner")
del peptides_aux_1, peptides_aux_2, peptides_aux_3, peptides_aux_4
new_train_peptides.reset_index(inplace=True)
print(f"New Train Peptides Dataframe shape has {new_train_peptides.shape[0]} rows and {new_train_peptides.shape[1]} columns")
display(new_train_peptides.head())
train_df = train_clinical_data.merge(new_train_peptides,
                                     on=["patient_id", "visit_id", "visit_month"],
                                     how="left")

train_df['upd23b_clinical_state_on_medication']= train_df['upd23b_clinical_state_on_medication'].replace('On', 1)
train_df['upd23b_clinical_state_on_medication']=train_df['upd23b_clinical_state_on_medication'].replace('Off', 0)
dft = train_df.astype(float)

New Train Peptides Dataframe shape has 1113 rows and 1897 columns


,patient_id,visit_id,visit_month,UniProt_0,UniProt_1,UniProt_2,UniProt_3,UniProt_4,UniProt_5,UniProt_6,...,PeptideAbundance_937,PeptideAbundance_938,PeptideAbundance_939,PeptideAbundance_940,PeptideAbundance_941,PeptideAbundance_942,PeptideAbundance_943,PeptideAbundance_944,PeptideAbundance_945,PeptideAbundance_946
0,55,55_0,0,0,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55,55_12,12,0,1,1,1,1,1,1,...,81764.2,168563.0,65762.6,29193.4,28536.1,19290.9,NaN,NaN,NaN,NaN
2,55,55_36,36,0,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55,55_6,6,0,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,942,942_12,12,0,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [831]:
le_cols_1 = [col for col in train_df.columns if 'UniProt' in col][0:config.MAX_PEPTIDES]
le_cols_2 = [col for col in train_df.columns if 'PeptideAbundance' in col][0:config.MAX_PEPTIDES]
TRAIN_COLUMNS = le_cols_1 + le_cols_2 + ["patient_id", "visit_month"]
TARGET_COLUMNS = ["updrs_1", "updrs_2", "updrs_3", "updrs_4" ] 

sub = (" ,upd23b_clinical_state_on_medication, visit_month" )

# === Filla NA ===
train_df.loc[:, train_df.columns.isin(TARGET_COLUMNS)] = train_df.loc[:, train_df.columns.isin(TARGET_COLUMNS)].fillna(0)
train_df.loc[:, train_df.columns.isin(le_cols_1)] = train_df.loc[:, train_df.columns.isin(le_cols_1)].fillna(-1)
train_df.loc[:, train_df.columns.isin(le_cols_2)] = train_df.loc[:, train_df.columns.isin(le_cols_2)].fillna(0)


train_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,UniProt_0,UniProt_1,...,PeptideAbundance_937,PeptideAbundance_938,PeptideAbundance_939,PeptideAbundance_940,PeptideAbundance_941,PeptideAbundance_942,PeptideAbundance_943,PeptideAbundance_944,PeptideAbundance_945,PeptideAbundance_946
0,55_0,55,0,10.0,6.0,15.0,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,0.0,NaN,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55_12,55,12,10.0,10.0,41.0,0.0,1.0,0.0,1.0,...,81764.2,168563.0,65762.6,29193.4,28536.1,19290.9,NaN,NaN,NaN,NaN


In [832]:
def clean_dataset(df):
    assert isinstance(df_train, pd.DataFrame), "df needs to be a pd.DataFrame"
    df_train.dropna(inplace=True)
    indices_to_keep = ~df_train.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df_train[indices_to_keep].astype(np.float64)

In [833]:
from sklearn.model_selection import train_test_split

X = train_df.loc[:, train_df.columns.isin(TRAIN_COLUMNS)]
y = train_df.loc[:, train_df.columns.isin(TARGET_COLUMNS)]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train

,patient_id,visit_month,UniProt_0,UniProt_1,UniProt_2,UniProt_3,UniProt_4,UniProt_5,UniProt_6,UniProt_7,...,PeptideAbundance_10,PeptideAbundance_11,PeptideAbundance_12,PeptideAbundance_13,PeptideAbundance_14,PeptideAbundance_15,PeptideAbundance_16,PeptideAbundance_17,PeptideAbundance_18,PeptideAbundance_19
23,942,54,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.00,0.0,0.00,0.00,0.00,0.000,0.00,0.0,0.0,0.0
1902,48928,0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,8003.39,30085.0,15254.20,6214.82,2136.18,109700.000,2626.59,45319.1,37778.8,17963.1
427,12755,72,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.00,0.0,0.00,0.00,0.00,0.000,0.00,0.0,0.0,0.0
554,14270,24,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,3135.18,32765.6,29745.00,104489.00,161911.00,51984.600,39041.00,25182.6,38559.1,24369.8
1420,33558,12,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,8421.27,31602.7,9581.53,4933.28,3535.82,84828.100,54903.80,51000.7,17841.7,223270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,40876,30,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.00,0.0,0.00,0.00,0.00,0.000,0.00,0.0,0.0,0.0
1095,26005,24,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,29962.90,18889.1,7374.08,3372.89,60165.10,885.721,37933.30,25426.3,13600.6,207573.0
1130,26809,0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,19796.70,11467.9,4265.60,6191.27,49932.00,822.622,26651.60,16038.6,159675.0,92164.5
1294,28818,84,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.00,0.0,0.00,0.00,0.00,0.000,0.00,0.0,0.0,0.0


In [834]:
# create an xgboost regression model
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

xgb_reg = MultiOutputRegressor(
    xgb.XGBRegressor(n_estimators=300, max_depth=15, objective='reg:squarederror')
)

xgb_reg = xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)

In [835]:
RAIN_COLUMNS = ["age", "sex", "test_time", "motor_UPDRS", "total_UPDRS"]
MODEL_PARAMS = {"objective": "reg:squarederror", "max_depth": 5, "n_estimators": 500, "seed": 42}

def process_train(train_data):
    train_df = pd.read_csv(train_data)
    X_train = train_df.loc[:, train_df.columns.isin(TRAIN_COLUMNS)]
    y_train = train_df["updrs"]
    xgb_reg = xgb.XGBRegressor(**MODEL_PARAMS)
    xgb_reg.fit(X_train, y_train)
    return xgb_reg

In [836]:
test_clinical_data = pd.read_csv(r"C:\Users\jshri\OneDrive\Desktop\supplemental_clinical_data.csv")
test_peptides = pd.read_csv(r"C:\Users\jshri\OneDrive\Desktop\test_peptides.csv")

In [837]:
def process_test(X_test, test_clinical_data, test_peptides):
    X_test = X_test.set_index(["patient_id", "visit_month"]) 
    y_pred = xgb_reg.predict(X_test)
    predictions = process_predictions(y_pred, X_test)
    return predictions

if 'patient_id' in X_test.columns:
    print("Column 'A' exists")
else:
    print("Column 'A' does not exist")

Column 'A' exists


In [838]:
def process_predictions(y_pred, X_test):
    predictions = pd.DataFrame({"patient_id": X_test["patient_id"],
                                "visit_month": X_test["visit_month"],
                                "prediction": y_pred})
    if "updrs" in X_test.columns:
        predictions["actual"] = X_test["updrs"]
        predictions["mse"] = (predictions["prediction"] - predictions["actual"]) ** 2
        predictions["mae"] = abs(predictions["prediction"] - predictions["actual"])
    return predictions

In [839]:
X_test = X_test.loc[:, 'visit_month']
X_test = X_test.values.ravel()
X_test.shape

(445,)

In [840]:
y_test = y_test.loc[:, 'updrs_1']
y_tes = y_test.values.ravel()
y_test.shape

(445,)

In [841]:
accuracy_score(y_test, X_test)

0.017977528089887642